In [1]:
import pandas as pd

# 패키지 로딩

In [5]:
train = pd.read_csv("data/titanic/train.csv", index_col = "PassengerId")
test = pd.read_csv("data/titanic/test.csv", index_col = "PassengerId")

print(train.shape)  # 잘 돌아갈 것인지 미리 체크하는 용도
print(test.shape)

train.head()
test.head()

(891, 11)
(418, 10)


,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [22]:
# 주기적인 테스트 용도

train.shape

assert train.shape == (891,13)  # 만약 이게 안맞으면 AssertionError가 뜨게 됨
assert test.shape == (418,10)

test.shape

(418, 10)

# 전처리

In [55]:
# train.loc[행, 열]

######################### 성별

train.loc[train["Sex"] == "male", "Sex_encode"] = 0
train.loc[train["Sex"] == "female", "Sex_encode"] = 1

test.loc[test["Sex"] == "male", "Sex_encode"] = 0
test.loc[test["Sex"] == "female", "Sex_encode"] = 1

# train.loc[train["Sex"] == "male", "Sex"] = 0
# ==> 이렇게 하면, "Sex"라는 기존 칼럼 값을 변경하게 됨
# ==> 그러나 원본은 그대로 두고 작업하는 게 좋으므로, 
#     보통은 "Sex_encode" 컬럼 새로 만들어서 작업

######################### Embarked

train.loc[train["Embarked"] == "C", "Embarked_encode"] = "France" 
train.loc[train["Embarked"] != "C", "Embarked_encode"] = "UK"

######################### SibSp, Parch

train.loc[(train["SibSp"] == 0) & (train["Parch"] == 0), "Familysize"] = 0
train["Familysize"] = train["SibSp"] + train["Parch"] + 1

test.loc[(train["SibSp"] == 0) & (test["Parch"] == 0), "Familysize"] = 0
test["Familysize"] = test["SibSp"] + test["Parch"] + 1

# train[""]

######################### Fare

train["Fare_fillin"] = train["Fare"]

train.loc[train["Fare_fillin"].isnull(), "Fare_fillin"] = 0
#train["Fare"].mean()

train.shape
train.head()

train[["Sex","Sex_encode","Embarked","Embarked_encode", "Familysize","Fare","Fare_fillin"]].head()

test["Fare_fillin"] = test["Fare"]

test.loc[test["Fare_fillin"].isnull(), "Fare_fillin"] = test["Fare"].mean()

# 평균 넣는 것, 평균 대신에 0이 들어갈 수도 있음

test[test["Fare"].isnull()]
# Fare가 비어있는 것을 보여주는

test[["Age","Fare"]].describe() # 2개씩 들어갈 수 있음


,Age,Fare
count,332.000000,417.000000
mean,30.272590,35.627188
std,14.181209,55.907576
min,0.170000,0.000000
25%,21.000000,7.895800
50%,27.000000,14.454200
75%,39.000000,31.500000
max,76.000000,512.329200


# One hot encoding

In [61]:
# Embarked encoding
# C == 0, S == 1, Q == 2
# 1 + S == Q ?
# S + S == Q ? 

# One hot encoding
# C = [1,0,0]
# S = [0,1,0]
# Q = [0,0,1]

# True == 1, False == 0
# C = [True,False,False]
# S = [False,True,False]
# Q = [False,False,True]

train["Embarked_C"] = (train["Embarked"] == "C")
train["Embarked_S"] = (train["Embarked"] == "S")
train["Embarked_Q"] = (train["Embarked"] == "Q")

print(train.shape)

test["Embarked_C"] = (test["Embarked"] == "C")
test["Embarked_S"] = (test["Embarked"] == "S")
test["Embarked_Q"] = (test["Embarked"] == "Q")

print(test.shape)


test.Embarked
test["Embarked"] # 열 선택할 때, 특수문자를 쓸 수 있는 방식

test[["Embarked", "Embarked_C", "Embarked_S", "Embarked_Q"]].head()
train[["Embarked", "Embarked_C", "Embarked_S", "Embarked_Q"]].head()

(891, 18)
(418, 16)


,Embarked,Embarked_C,Embarked_S,Embarked_Q
PassengerId,,,,
1,S,False,True,False
2,C,True,False,False
3,S,False,True,False
4,S,False,True,False
5,S,False,True,False


# 의사결정트리 - 학습

feature 정답에 도움을 주는 것, label 정답

In [109]:
feature_names = [ "Pclass", "Sex_encode", "Fare_fillin"
                , "Embarked_C", "Embarked_S", "Embarked_Q"]

feature_names2 = [ "Pclass", "Sex_encode", "Fare_fillin"
                , "Embarked_C", "Embarked_S", "Embarked_Q"
                 , "Familysize"]

feature_names3 = [ "Pclass", "Sex_encode", "Fare_fillin"
                , "Embarked_C", "Embarked_S", "Embarked_Q"
                 , "Familysize", "Age", "Fare"]   # 여기에서 Age 부분 같은게 비어 있으면 적용할 수 없음

X_train = train[feature_names]
X_test = test[feature_names]

X_train.head()
X_test.head()

#assert X_train.shape == (891,6)
#assert X_test.shape == (418,6)

# 매트릭스(행렬)은 대문자로 구현
# 1개 행으로만 구성된 것은 벡터로 소문자로 구현
# 별 상관없음

,Pclass,Sex_encode,Fare_fillin,Embarked_C,Embarked_S,Embarked_Q
PassengerId,,,,,,
892,3,0.0,7.8292,False,False,True
893,3,1.0,7.0000,False,True,False
894,2,0.0,9.6875,False,False,True
895,3,0.0,8.6625,False,True,False
896,3,1.0,12.2875,False,True,False


# 의사결정 트리 (예측)

In [110]:
lable_name = "Survived"

y_train = train[lable_name] # y_test에는 "Survived"가 없으므로 넣으면 에러

print(y_train.shape)
y_train.head()

(891,)


PassengerId
1    0
2    1
3    1
4    1
5    0
Name: Survived, dtype: int64

In [112]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(max_depth = 5)
model

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [113]:
# model.fit (피처, 레이블)
# model.predic (피처)

model.fit(X_train, y_train)
predictions = model.predict(X_test)

print (predictions.shape)
predictions[0:5]

# 아래 결과에서 dtype = int64라고 나오는 것은 쥬피터 노트북 버전 차이

(418,)


array([0, 0, 0, 0, 1], dtype=int64)

# 제출하기

In [115]:
submit = pd.read_csv("data/titanic/gender_submission.csv", index_col="PassengerId")

submit["Survived"] = predictions

print(submit.shape)
submit.head()

(418, 1)


,Survived
PassengerId,
892,0
893,0
894,0
895,0
896,1


In [118]:
submit.to_csv("data/Titanic/decision-tree.csv")